In [ ]:
import libsbml
import importlib
import amici
import amici.plotting
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import scipy.stats

from gm import gm
from gm_Prep import gm_Prep

In [ ]:
# np.random.seed(1337) # set a random seed number or not

In [ ]:
flagD = 0 # set as deterministic(1) or stochastic(0)
th = 24 # set simulation time in hours 

ts = 30 # time step
NSteps = th*3600/ts # number of exchanges between stochastic module and deterministic module
NSteps = int(NSteps)

# SBML model we want to import
sbml_file = 'SPARCED_Brep.xml'
# Name of the model that will also be the name of the python module
model_name = sbml_file[0:-4] 
# Directory (name) to which the generated model code will be written
model_output_dir = model_name

# get the model path and instance
sys.path.insert(0, os.path.abspath(model_output_dir))
model_module = importlib.import_module(model_name)
model = model_module.getModel()
solver = model.getSolver() # Create solver instance
solver.setMaxSteps = 1e10
model.setTimepoints(np.linspace(0,ts)) # set timepoints

# Read-in experimental parameters
kGsRead = pd.read_csv('OmicsData.txt',header=0,index_col=0,sep="\t")
gExp_mpc = np.float64(kGsRead.values[:,0])
mExp_mpc = np.float64(kGsRead.values[:,1])
kGin = np.float64(kGsRead.values[:,2])
kGac = np.float64(kGsRead.values[:,3])
kTCleak = np.float64(kGsRead.values[:,4])
kTCmaxs = np.float64(kGsRead.values[:,5])
kTCd = np.float64(kGsRead.values[:,6])
kmRNAs = kGsRead.values[:,7] # parameter names corresponding to the mRNA amounts in translation ratelaws 

# Read-in the activators matrix and assign concentrations of activators
TAsRead = pd.read_csv('TAs.csv',header=0,index_col=0)
TAs0 = np.float64(TAsRead.values)
# Read-in the repressors matrix and assign concentrations of repressors
TRsRead = pd.read_csv('TRs.csv',header=0,index_col=0)
TRs0 = np.float64(TRsRead.values)

In [ ]:
startTime = datetime.now()
Vn = 1.75E-12
Vc = 5.25E-12

spdata0 = pd.read_csv('Species_Brep.txt',header=0,index_col=0,sep="\t")
spdata = np.double(spdata0.values[:,1])

genedata0, GenePositionMatrix, AllGenesVec, xgac_mpc_D, xgin_mpc_D, xgac_mpc, xgin_mpc, kTCleak2 \
= gm_Prep(flagD, gExp_mpc, mExp_mpc, kGin, kGac, kTCleak, kTCmaxs, kTCd)

# Create empty species, gene, and time point matrices
tout_all = np.arange(0,th*3600+1,30)
xoutS_all = np.zeros(shape=(NSteps+1,len(spdata)))
xoutS_all[0,:] = spdata
xoutG_all = np.zeros(shape=(NSteps+1,len(genedata0)))
xoutG_all[0,:] = genedata0
    
for qq in range(NSteps):
    genedata,AllGenesVec = gm(flagD,ts,xoutG_all[qq,:],xoutS_all[qq,:],Vn,Vc,kGin,kGac, \
                               kTCmaxs,kTCleak,kTCd,AllGenesVec,GenePositionMatrix,TAs0,TRs0)
    for ii,kk in enumerate(kmRNAs):
        model.setFixedParameterById(kk,genedata[ii+282]*(1E9/(Vc*6.023E+23)))
    model.setInitialStates(xoutS_all[qq,:])
    rdata = amici.runAmiciSimulation(model, solver)  # Run deterministic module simulation
    xoutS_all[qq+1,:] = rdata['x'][-1,:] # Next time point concentrations
    xoutG_all[qq+1,:] = genedata # Next time point gene states and mRNA numbers
    if rdata['x'][-1,103] < rdata['x'][-1,105]:
        print('Apoptosis happened')
        break
print(datetime.now() - startTime)

namexlsfile = 'Fig2B_24hr_stoc_'    
condsSDF = pd.DataFrame(data=xoutS_all,columns=[ele for ele in model.getStateIds()]) 
condsSDF.to_csv(namexlsfile+'S'+'.txt',sep="\t")   
condsSDF = None
condsGDF = pd.DataFrame(data=xoutG_all) 
condsGDF.to_csv(namexlsfile+'G'+'.txt',sep="\t")   
condsGDF = None

In [ ]:
plt.figure(figsize=(7, 4))
plt.plot(tout_all/3600.0,xoutG_all[:,115],'b',linewidth=4)
plt.xlabel('Time (hrs)')
plt.ylabel('MAPK1 act genes', multialignment='center')
plt.grid(True)
plt.xlim(0, 24)
plt.xticks(np.arange(0,24,step=4))
plt.savefig('Fig2B_g1.png')

plt.figure(figsize=(7, 4))
plt.plot(tout_all/3600.0,xoutG_all[:,116],'r',linewidth=4)
plt.xlabel('Time (hrs)')
plt.ylabel('MAPK3 act genes', multialignment='center')
plt.grid(True)
plt.xlim(0, 24)
plt.xticks(np.arange(0,24,step=4))
plt.savefig('Fig2B_g2.png')

In [ ]:
plt.figure(figsize=(7, 4))
plt.plot(tout_all/3600.0,xoutG_all[:,397],'b',linewidth=4)
plt.xlabel('Time (hrs)')
plt.ylabel('MAPK1 mRNAs', multialignment='center')
plt.grid(True)
plt.xlim(0, 24)
plt.xticks(np.arange(0,24,step=4))
plt.savefig('Fig2B_m1.png')

plt.figure(figsize=(7, 4))
plt.plot(tout_all/3600.0,xoutG_all[:,398],'r',linewidth=4)
plt.xlabel('Time (hrs)')
plt.ylabel('MAPK3 mRNAs', multialignment='center')
plt.grid(True)
plt.xlim(0, 24)
plt.xticks(np.arange(0,24,step=4))
plt.savefig('Fig2B_m2.png')

In [ ]:
tERKinds = [150,154,225,226,227,674,675,715,716,717,728,729,732,735,736,737,739,741,749,755,756,757]

plt.figure(figsize=(7, 4))
plt.plot(tout_all/3600.0,xoutS_all[:,tERKinds].sum(axis=1),'k',linewidth=4)
plt.xlabel('Time (hrs)')
plt.ylabel('total ERK (nM)', multialignment='center')
plt.grid(True)
plt.xlim(0, 24)
plt.xticks(np.arange(0,25,step=4))
plt.savefig('Fig2B_tERK.png')